<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/English/BERT_2k20_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# d=[]
# while(1):
#   d.append('1111111111111111111111111111')

In [0]:
!pip install focal-loss

In [0]:
!pip install ktrain

In [0]:
!pip install "tensorflow_gpu>=2.0.0"

In [0]:
!pip install tensorflow-addons

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [0]:
import ktrain
import pandas as pd
import numpy as np
# import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Sequential,Model
# from tensorflow.keras.layers import Embedding,Dense,Dropout,Flatten, Input
# from tensorflow.keras.optimizers import RMSprop,Adam,Adadelta

# Load data

In [0]:
train=pd.read_csv( '/content/drive/My Drive/dataset/OffenseEval2020/data/English/task_a_distant.tsv',sep="\t")


In [0]:
len(train)

In [0]:
train=train.iloc[0:int(len(train)/3),:]
len(train)

In [0]:
val=pd.read_csv( '/content/drive/My Drive/dataset/OLID/data/olid-training-v1.0.tsv',sep="\t")

val.head()

In [0]:
test=pd.read_csv( '/content/drive/My Drive/dataset/OffenseEval2020/data/English/test_a_tweets.tsv',sep="\t")
test.head()

In [0]:
y_train=train['average'].values
y_train[y_train>0.5]=1
y_train[y_train<0.5]=0
y_train=y_train.astype('int')


In [0]:
train=train['text']
train.head()

In [0]:
y_val=val['subtask_a']
val=val['tweet']
val.head()

In [0]:
ids=test['id']
test=test['tweet']
test.head()

In [0]:

train=train.str.replace('\d+', '')
train=train.str.replace('@USER', "") 
train=train.str.replace('URL', '')
train= train.str.lower()

train.head()

In [0]:

val=val.str.replace('\d+', '')
val=val.str.replace('@USER', "") 
val=val.str.replace('URL', '')
val= val.str.lower()
val.head()

In [0]:

test=test.str.replace('\d+', '')
test=test.str.replace('@USER', "") 
test=test.str.replace('URL', '')
test= test.str.lower()
test.head()

#Label Encoding

In [0]:
len(train),len(val),len(test)

In [0]:
le=LabelEncoder()
y_val=le.fit_transform(y_val)
print(len(y_val))

In [0]:
import gc
gc.collect()

#BERT

In [0]:
from focal_loss import BinaryFocalLoss

In [0]:

import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=100,classes=[0,1])
trn = t.preprocess_train(train, y_train)
val = t.preprocess_test(val, y_val)
model = t.get_classifier()


In [0]:
import tensorflow_addons as tfa
opt=tfa.optimizers.LAMB()	

In [0]:
model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=opt, metrics=['accuracy'])

In [0]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=64)

In [0]:
# learner.lr_find(show_plot=True, max_epochs=2)

In [0]:
learner.fit_onecycle(10e-5, 1)
#learner.autofit(10e-4, 10)


In [0]:
# learner.validate(class_names=t.get_classes())

In [0]:
# from sklearn.metrics import f1_score
# predictor = ktrain.get_predictor(learner.model, preproc=t)
# y_pred=predictor.predict(x_val.tolist(),return_proba=True)

# #y_pred = learner.model.predict(x_test, verbose=0)


In [0]:
# f1_score = f1_score(y_val, y_pred,average='macro')